In [0]:
from pyspark.sql.functions import col
from pyspark.sql.types import IntegerType, DoubleType, BooleanType, DateType
from pyspark.sql import functions as F

In [0]:
configs = {
    "fs.azure.account.auth.type": "OAuth",
    "fs.azure.account.oauth.provider.type": "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider",
    "fs.azure.account.oauth2.client.id": "",
    "fs.azure.account.oauth2.client.secret": '',
    "fs.azure.account.oauth2.client.endpoint": "https://login.microsoftonline.com/tenant_id/oauth2/token"
}

dbutils.fs.mount(
    source="abfss://tokyo-olympic-data@ertantokyoolympicdata.dfs.core.windows.net",
    mount_point="/mnt/tokyoolympic",
    extra_configs=configs
)

Out[2]: True

In [0]:
%fs
ls "/mnt/tokyoolympic"

path,name,size,modificationTime
dbfs:/mnt/tokyoolympic/raw-data/,raw-data/,0,1732107131000
dbfs:/mnt/tokyoolympic/transformed-data/,transformed-data/,0,1732107146000


In [0]:
athletes = spark.read.format("csv").load("/mnt/tokyoolympic/raw-data/athletes.csv",header=True)
coaches = spark.read.format("csv").load("/mnt/tokyoolympic/raw-data/coaches.csv",header=True)
entriesgender = spark.read.format("csv").load("/mnt/tokyoolympic/raw-data/entriesgender.csv",header=True)
medals = spark.read.format("csv").option("inferSchema", "true").load("/mnt/tokyoolympic/raw-data/medals.csv",header=True)
teams = spark.read.format("csv").option("inferSchema", "true").load("/mnt/tokyoolympic/raw-data/teams.csv",header=True)

In [0]:
athletes.show()

+--------------------+--------------------+-------------------+
|          personname|             country|         discipline|
+--------------------+--------------------+-------------------+
|     AALERUD Katrine|              Norway|       Cycling Road|
|         ABAD Nestor|               Spain|Artistic Gymnastics|
|   ABAGNALE Giovanni|               Italy|             Rowing|
|      ABALDE Alberto|               Spain|         Basketball|
|       ABALDE Tamara|               Spain|         Basketball|
|           ABALO Luc|              France|           Handball|
|        ABAROA Cesar|               Chile|             Rowing|
|       ABASS Abobakr|               Sudan|           Swimming|
|    ABBASALI Hamideh|Islamic Republic ...|             Karate|
|       ABBASOV Islam|          Azerbaijan|          Wrestling|
|        ABBINGH Lois|         Netherlands|           Handball|
|         ABBOT Emily|           Australia|Rhythmic Gymnastics|
|       ABBOTT Monica|United States of .

In [0]:
athletes.printSchema()

root
 |-- personname: string (nullable = true)
 |-- country: string (nullable = true)
 |-- discipline: string (nullable = true)



In [0]:
coaches.show()

+--------------------+--------------------+-----------------+--------+
|                name|             country|       discipline|   event|
+--------------------+--------------------+-----------------+--------+
|     ABDELMAGID Wael|               Egypt|         Football|    null|
|           ABE Junya|               Japan|       Volleyball|    null|
|       ABE Katsuhiko|               Japan|       Basketball|    null|
|        ADAMA Cherif|      CÃ´te d'Ivoire|         Football|    null|
|          AGEBA Yuya|               Japan|       Volleyball|    null|
|AIKMAN Siegfried ...|               Japan|           Hockey|     Men|
|       AL SAADI Kais|             Germany|           Hockey|     Men|
|       ALAMEDA Lonni|              Canada|Baseball/Softball|Softball|
|     ALEKNO Vladimir|Islamic Republic ...|       Volleyball|     Men|
|     ALEKSEEV Alexey|                 ROC|         Handball|   Women|
|ALLER CARBALLO Ma...|               Spain|       Basketball|    null|
|     

In [0]:
coaches.printSchema()

root
 |-- name: string (nullable = true)
 |-- country: string (nullable = true)
 |-- discipline: string (nullable = true)
 |-- event: string (nullable = true)



In [0]:
entriesgender.show()

+--------------------+------+----+-----+
|          discipline|female|male|total|
+--------------------+------+----+-----+
|      3x3 Basketball|    32|  32|   64|
|             Archery|    64|  64|  128|
| Artistic Gymnastics|    98|  98|  196|
|   Artistic Swimming|   105|   0|  105|
|           Athletics|   969|1072| 2041|
|           Badminton|    86|  87|  173|
|   Baseball/Softball|    90| 144|  234|
|          Basketball|   144| 144|  288|
|    Beach Volleyball|    48|  48|   96|
|              Boxing|   102| 187|  289|
|        Canoe Slalom|    41|  41|   82|
|        Canoe Sprint|   123| 126|  249|
|Cycling BMX Frees...|    10|   9|   19|
|  Cycling BMX Racing|    24|  24|   48|
|Cycling Mountain ...|    38|  38|   76|
|        Cycling Road|    70| 131|  201|
|       Cycling Track|    90|  99|  189|
|              Diving|    72|  71|  143|
|          Equestrian|    73| 125|  198|
|             Fencing|   107| 108|  215|
+--------------------+------+----+-----+
only showing top

In [0]:
entriesgender.printSchema()

root
 |-- discipline: string (nullable = true)
 |-- female: string (nullable = true)
 |-- male: string (nullable = true)
 |-- total: string (nullable = true)



In [0]:
entriesgender = entriesgender.withColumn("female", col("female").cast(IntegerType()))\
    .withColumn("male", col("male").cast(IntegerType()))\
        .withColumn("total", col("total").cast(IntegerType()))

In [0]:
entriesgender.printSchema()

root
 |-- discipline: string (nullable = true)
 |-- female: integer (nullable = true)
 |-- male: integer (nullable = true)
 |-- total: integer (nullable = true)



In [0]:
medals.show()

+----+--------------------+----+------+------+-----+-------------+
|rank|         teamcountry|gold|silver|bronze|total|rank_by_total|
+----+--------------------+----+------+------+-----+-------------+
|   1|United States of ...|  39|    41|    33|  113|            1|
|   2|People's Republic...|  38|    32|    18|   88|            2|
|   3|               Japan|  27|    14|    17|   58|            5|
|   4|       Great Britain|  22|    21|    22|   65|            4|
|   5|                 ROC|  20|    28|    23|   71|            3|
|   6|           Australia|  17|     7|    22|   46|            6|
|   7|         Netherlands|  10|    12|    14|   36|            9|
|   8|              France|  10|    12|    11|   33|           10|
|   9|             Germany|  10|    11|    16|   37|            8|
|  10|               Italy|  10|    10|    20|   40|            7|
|  11|              Canada|   7|     6|    11|   24|           11|
|  12|              Brazil|   7|     6|     8|   21|          

In [0]:
medals.printSchema()

root
 |-- rank: integer (nullable = true)
 |-- teamcountry: string (nullable = true)
 |-- gold: integer (nullable = true)
 |-- silver: integer (nullable = true)
 |-- bronze: integer (nullable = true)
 |-- total: integer (nullable = true)
 |-- rank_by_total: integer (nullable = true)



In [0]:
teams.printSchema()

root
 |-- teamname: string (nullable = true)
 |-- discipline: string (nullable = true)
 |-- country: string (nullable = true)
 |-- event: string (nullable = true)



In [0]:
# find the top 5 countries with the highest amount of gold medals
top_gold = (medals.groupBy("teamcountry").agg(F.sum("gold").alias("total_gold")).orderBy("total_gold", ascending=False).limit(5))

top_gold.show()

+--------------------+----------+
|         teamcountry|total_gold|
+--------------------+----------+
|United States of ...|        39|
|People's Republic...|        38|
|               Japan|        27|
|       Great Britain|        22|
|                 ROC|        20|
+--------------------+----------+



In [0]:
# Calculate the average number of entries by gender for each discipline
avg_entries_by_gender = entriesgender.withColumn("avg_female", entriesgender['female'] / entriesgender['total'])\
    .withColumn("avg_male", entriesgender['male'] / entriesgender['total'])

avg_entries_by_gender.show()

+--------------------+------+----+-----+-------------------+-------------------+
|          discipline|female|male|total|         avg_female|           avg_male|
+--------------------+------+----+-----+-------------------+-------------------+
|      3x3 Basketball|    32|  32|   64|                0.5|                0.5|
|             Archery|    64|  64|  128|                0.5|                0.5|
| Artistic Gymnastics|    98|  98|  196|                0.5|                0.5|
|   Artistic Swimming|   105|   0|  105|                1.0|                0.0|
|           Athletics|   969|1072| 2041| 0.4747672709456149| 0.5252327290543851|
|           Badminton|    86|  87|  173|0.49710982658959535| 0.5028901734104047|
|   Baseball/Softball|    90| 144|  234|0.38461538461538464| 0.6153846153846154|
|          Basketball|   144| 144|  288|                0.5|                0.5|
|    Beach Volleyball|    48|  48|   96|                0.5|                0.5|
|              Boxing|   102

In [0]:
athletes.repartition(1).write.mode("overwrite").option("header", "true").csv("/mnt/tokyoolympic/transformed-data/athletes")
coaches.repartition(1).write.mode("overwrite").option("header", "true").csv("/mnt/tokyoolympic/transformed-data/coaches")
entriesgender.repartition(1).write.mode("overwrite").option("header", "true").csv("/mnt/tokyoolympic/transformed-data/entriesgender")
medals.repartition(1).write.mode("overwrite").option("header", "true").csv("/mnt/tokyoolympic/transformed-data/medals") 
teams.repartition(1).write.mode("overwrite").option("header", "true").csv("/mnt/tokyoolympic/transformed-data/teams")   